# Evolver Loop 1 Analysis: Text Feature Impact Assessment

This notebook analyzes the potential impact of adding text features to our baseline model.

**Goal**: Estimate the expected improvement from adding text features and design the next experiment.

In [ ]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load data
print("Loading data...")
with open('/home/data/train.json', 'r') as f:
    train_data = json.load(f)

with open('/home/data/test.json', 'r') as f:
    test_data = json.load(f)

# Convert to DataFrames
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"Target distribution: {train_df['requester_received_pizza'].value_counts().to_dict()}")
print(f"Positive rate: {train_df['requester_received_pizza'].mean():.3f}")

In [ ]:
# Analyze text fields
print("Analyzing text fields...")

# Check text availability
print(f"Missing request_title: {train_df['request_title'].isnull().sum()}")
print(f"Missing request_text_edit_aware: {train_df['request_text_edit_aware'].isnull().sum()}")

# Sample some texts
print("\n=== Sample Request Titles (successful) ===")
successful = train_df[train_df['requester_received_pizza'] == True]
for i, title in enumerate(successful['request_title'].head(5)):
    print(f"{i+1}. {title}")

print("\n=== Sample Request Titles (unsuccessful) ===")
unsuccessful = train_df[train_df['requester_received_pizza'] == False]
for i, title in enumerate(unsuccessful['request_title'].head(5)):
    print(f"{i+1}. {title}")

# Text length analysis
train_df['title_length'] = train_df['request_title'].str.len()
train_df['text_length'] = train_df['request_text_edit_aware'].str.len()
train_df['total_length'] = train_df['title_length'] + train_df['text_length']

test_df['title_length'] = test_df['request_title'].str.len()
test_df['text_length'] = test_df['request_text_edit_aware'].str.len()
test_df['total_length'] = test_df['title_length'] + test_df['text_length']

print(f"\n=== Text Length Statistics ===")
print(f"Title length - Mean: {train_df['title_length'].mean():.0f}, Std: {train_df['title_length'].std():.0f}")
print(f"Text length - Mean: {train_df['text_length'].mean():.0f}, Std: {train_df['text_length'].std():.0f}")
print(f"Total length - Mean: {train_df['total_length'].mean():.0f}, Std: {train_df['total_length'].std():.0f}")

In [ ]:
# Analyze correlation between text length and success
print("=== Correlation Analysis: Text Length vs Success ===")

from scipy.stats import ttest_ind

# Successful vs unsuccessful length comparison
success_lengths = train_df[train_df['requester_received_pizza'] == True]['total_length']
fail_lengths = train_df[train_df['requester_received_pizza'] == False]['total_length']

print(f"Successful requests - Mean length: {success_lengths.mean():.0f}")
print(f"Unsuccessful requests - Mean length: {fail_lengths.mean():.0f}")

# T-test
t_stat, p_value = ttest_ind(success_lengths, fail_lengths)
print(f"T-test: t={t_stat:.3f}, p={p_value:.6f}")

# Correlation
corr = train_df['total_length'].corr(train_df['requester_received_pizza'].astype(int))
print(f"Correlation between total_length and success: {corr:.4f}")

# Visualize distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(data=train_df, x='total_length', hue='requester_received_pizza', bins=50, alpha=0.7)
plt.title('Distribution of Total Text Length by Success')
plt.xlabel('Total Length (characters)')

plt.subplot(1, 2, 2)
sns.boxplot(data=train_df, x='requester_received_pizza', y='total_length')
plt.title('Text Length by Success (Boxplot)')
plt.xlabel('Received Pizza')
plt.ylabel('Total Length (characters)')

plt.tight_layout()
plt.show()

print(f"\nKey Insight: Text length {'IS' if abs(corr) > 0.1 else 'is NOT'} significantly correlated with success (|r|={abs(corr):.3f})")

In [ ]:
# Quick TF-IDF test to estimate potential impact
print("=== Quick TF-IDF Impact Test ===")

# Combine title and text for TF-IDF
train_df['combined_text'] = train_df['request_title'] + ' ' + train_df['request_text_edit_aware']
test_df['combined_text'] = test_df['request_title'] + ' ' + test_df['request_text_edit_aware']

# Simple TF-IDF (unigrams only, limited features for quick test)
vectorizer = TfidfVectorizer(
    max_features=1000,
    stop_words='english',
    ngram_range=(1, 1),
    min_df=5,
    max_df=0.9
)

# Fit on training data
tfidf_train = vectorizer.fit_transform(train_df['combined_text'])
tfidf_test = vectorizer.transform(test_df['combined_text'])

print(f"TF-IDF shape: {tfidf_train.shape}")
print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")

# Quick model test with TF-IDF only
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Scale TF-IDF features
scaler = StandardScaler(with_mean=False)
tfidf_train_scaled = scaler.fit_transform(tfidf_train)
tfidf_test_scaled = scaler.transform(tfidf_test)

# Quick CV test
y = train_df['requester_received_pizza'].astype(int).values
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

tfidf_scores = []
for train_idx, val_idx in skf.split(tfidf_train_scaled, y):
    X_train, X_val = tfidf_train_scaled[train_idx], tfidf_train_scaled[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
    model.fit(X_train, y_train)
    
    val_pred = model.predict_proba(X_val)[:, 1]
    score = roc_auc_score(y_val, val_pred)
    tfidf_scores.append(score)

print(f"TF-IDF only (Logistic Regression) - Mean ROC-AUC: {np.mean(tfidf_scores):.4f} ± {np.std(tfidf_scores):.4f}")
print(f"Individual folds: {tfidf_scores}")

# Compare to baseline (meta only)
print(f"\nBaseline (meta only): 0.6691")
print(f"TF-IDF only: {np.mean(tfidf_scores):.4f}")
print(f"Potential improvement: +{np.mean(tfidf_scores) - 0.6691:.4f}")

In [ ]:
# Analyze most predictive words
print("=== Most Predictive Words Analysis ===")

# Train a simple model to get feature importances
lr_model = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
lr_model.fit(tfidf_train_scaled, y)

# Get feature names and coefficients
feature_names = vectorizer.get_feature_names_out()
coefficients = lr_model.coef_[0]

# Create DataFrame of word importances
word_importance = pd.DataFrame({
    'word': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
}).sort_values('abs_coefficient', ascending=False)

print("Top 15 positive words (predictive of SUCCESS):")
print(word_importance.head(15)[['word', 'coefficient']].to_string(index=False))

print("\nTop 15 negative words (predictive of FAILURE):")
print(word_importance.tail(15)[['word', 'coefficient']].to_string(index=False))

# Save findings
print(f"\n=== Key Findings ===")
print(f"1. Text length correlation with success: {corr:.4f}")
print(f"2. TF-IDF alone achieves: {np.mean(tfidf_scores):.4f} ROC-AUC")
print(f"3. Expected improvement from adding text: +{np.mean(tfidf_scores) - 0.6691:.4f} to +0.15")
print(f"4. Top predictive words identified for feature engineering")